# **French Motor Third-Party Liability Claims**

Daniel Meier and Jürg Schelldorfer, with support from Christian Lorentzen, Friedrich Loser, Michael Mayer, Mario V. Wüthrich and [Mirai Solutions GmbH](https://mirai-solutions.ch/).

2021-15-10

# Introduction

This notebook was created for the course "Deep Learning with Actuarial Applications in R" of the Swiss Association of Actuaries (https://www.actuaries.ch/).

This notebook serves as companion to the tutorial “Case Study: French Motor Third-Party Liability Claims”, available on [SSRN](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3164764).

The code is similar to the code used in above tutorial and combines the raw R code in the scripts, available on [GitHub](https://github.com/JSchelldorfer/ActuarialDataScience/tree/master/1%20-%20French%20Motor%20Third-Party%20Liability%20Claims) along with some more comments. Please refer to the tutorial for explanations.

Note that the results might vary depending on the R and Python package versions, see last section for the result of `sessionInfo()` and corresponding info on the Python setup.

# Data Preparation

The tutorial uses the French MTPL data set available on [openML (ID 41214)](https://www.openml.org/d/41214).

## Load packages and data


In [ ]:
# library(mgcv)
library(dplyr)
library(tibble)
library(ggplot2)
library(splitTools)


In [ ]:
# plotting parameters in R Markdown notebook
knitr::opts_chunk$set(fig.width = 9, fig.height = 9)
# plotting parameters in Jupyter notebook
library(repr)  # only needed for Jupyter notebook
options(repr.plot.width = 9, repr.plot.height = 9)


## Set global parameters



In [ ]:
options(encoding = 'UTF-8')



In [ ]:
# set seed to obtain best reproducibility. note that the underlying architecture may affect results nonetheless, so full reproducibility cannot be guaranteed across different platforms.
seed <- 100


## Helper functions

Subsequently, for ease of reading, we provide all the helper functions which are used in this tutorial in this section.


In [ ]:
summarize <- function(...) suppressMessages(dplyr::summarize(...))



In [ ]:
load_data <- function(file) {
  load(file.path("../0_data/", file), envir = parent.frame(1))
}


In [ ]:
# Poisson deviance
PoissonDeviance <- function(pred, obs) {
  200 * (sum(pred) - sum(obs) + sum(log((obs / pred)^(obs)))) / length(pred)
}


In [ ]:
plot_freq <- function(test, xvar, title, model, mdlvariant) {
  out <- test %>% group_by(!!sym(xvar)) %>% summarize(obs = sum(ClaimNb) / sum(Exposure),
                                                      pred = sum(!!sym(mdlvariant)) / sum(Exposure))
  
  ggplot(out, aes(x = !!sym(xvar), group = 1)) + geom_point(aes(y = pred, colour = model)) +
    geom_point(aes(y = obs, colour = "observed")) +
    geom_line(aes(y = pred, colour = model), linetype = "dashed") +
    geom_line(aes(y = obs, colour = "observed"), linetype = "dashed") +
    ylim(0, 0.35) + labs(x = xvar, y = "frequency", title = title) +
    theme(legend.position = "bottom")
}


## Load data

We consider the data `freMTPL2freq` included in the R package `CASdatasets` for claim frequency modeling. This data comprises a French motor third-party liability (MTPL) insurance portfolio with corresponding claim counts observed in one accounting year.
We do not incorporate claim sizes which would also be available through freMTPL2sev.

As the current package version provides a slightly amended dataset, we use an older dataset available on [openML (ID 41214)](https://www.openml.org/d/41214). Before we can use this data set we need to do some data cleaning. It has been pointed out by [F. Loser](https://www.kaggle.com/floser/comparing-claims-fremtpl2freq-sev/execution) that some claim counts do not seem to be correct. Hence, we use the pre-processing of the data described in the book ["Statistical Foundations of Actuarial Learning and its Applications"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3822407) in Appendix A.1. This pre-processed data can be downloaded from the course GitHub page [here](https://github.com/JSchelldorfer/DeepLearningWithActuarialApplications).


In [ ]:
load_data("freMTPL2freq.RData")



## General data preprocessing

A priori, there is not sufficient information about this data to do a sensible decision about the best consideration of the exposure measure, either as feature or as offset. In the following we treat the exposure always as an offset.

Data preprocessing includes a couple of transformations. We ensure that $\texttt{ClaimNb}$ is an integer, $\texttt{VehAge}$, $\texttt{DrivAge}$ and $\texttt{BonusMalus}$ have been capped for the plots at age 20, age 90 and bonus-malus level 150, respectively, to improve visualization. $\texttt{Density}$ is logarithmized and $\texttt{VehGas}$ is a categorical variable. We leave away the rounding used in the first notebook, which were mainly used for nicer visualizations of the data.

We are adding a group_id identifying rows possibly referring to the same policy. Respecting group_id in data splitting techniques (train/test, cross-validation) is essential. This is different to the tutorial where another splitting has been used. As a consequence, the figures in this notebook do not match the figures in the tutorial, but the conclusions drawn are the same.

In addition to the previous tutorial, we decide to truncate the $\texttt{ClaimNb}$ and the $\texttt{Exposure} in order to correct for unreasonable data entries and simplifications for the modeling part.


In [ ]:
# Grouping id
distinct <- freMTPL2freq %>% 
  distinct_at(vars(-c(IDpol, Exposure, ClaimNb))) %>% 
  mutate(group_id = row_number())


In [ ]:
dat <- freMTPL2freq %>% 
  left_join(distinct) %>% 
  mutate(ClaimNb = pmin(as.integer(ClaimNb), 4),
         VehAge = pmin(VehAge, 20),
         DrivAge = pmin(DrivAge, 90),
         BonusMalus = pmin(BonusMalus, 150),
         Density = round(log(Density), 2),
         VehGas = factor(VehGas),
         Exposure = pmin(Exposure, 1))


In [ ]:
# Group sizes of suspected clusters
table(table(dat[, "group_id"]))


## Feature pre-processing for generalized linear models

As previously mentioned, typically features $x_i$ need pre-processing before being used for a specific model. In our Poisson GLM the regression function is modeled by a log-linear shape in the continuous feature components. From the marginal empirical frequency plots in the previous file we see that such a log-linear form is not always appropriate. We make the following choices here:

- $\texttt{Area}$: we choose a continuous (log-linear) feature component for $\{A,...,F\}$ $\mapsto$ $\{1,...,6\}$
- $\texttt{VehPower}$: we choose a categorical feature component where we merge vehicle power groups bigger and equal to 9 (totally 6 classes)
- $\texttt{VehAge}$: we build 3 categorical classes $[0,1), [1,10], (10,\infty)$
- $\texttt{DrivAge}$: we build 7 categorical classes $[18,21), [21,26), [26,31), [31,41), [41,51), [51,71),
[71,\infty)$
- $\texttt{BonusMalus}$: continuous log-linear feature component (we cap at value 150)
- $\texttt{VehBrand}$: categorical feature component (totally 11 classes)
- $\texttt{VehGas}$: binary feature component;
- $\texttt{Density}$: log-density is chosen as continuous log-linear feature component (note that we
have very small volumes for small log-densities)
- $\texttt{Region}$: categorical feature component (totally 22 classes)

Thus, we consider 3 continuous feature components ($\texttt{Area}$, $\texttt{BonusMalus}$, $\texttt{log-Density}$), 1 binary feature component ($\texttt{VehGas}$) and 5 categorical feature components ($\texttt{VehPower}$, $\texttt{VehAge}$, $\texttt{DrivAge}$, $\texttt{VehBrand}$, $\texttt{Region}$). The categorical classes for $\texttt{VehPower}$, $\texttt{VehAge}$ and $\texttt{DrivAge}$ have been done based on expert opinion, only. This expert opinion has tried to find homogeneity within class labels (levels) and every class label should receive a sufficient volume (of observations). We could also make a data-driven choice by using a (marginal) regression tree for different feature components, see references in the tutorial.


In [ ]:
dat2 <- dat %>% mutate(
  AreaGLM = as.integer(Area),
  VehPowerGLM = as.factor(pmin(VehPower, 9)),
  VehAgeGLM = cut(VehAge, breaks = c(-Inf, 0, 10, Inf), labels = c("1","2","3")),
  DrivAgeGLM = cut(DrivAge, breaks = c(-Inf, 20, 25, 30, 40, 50, 70, Inf), labels = c("1","2","3","4","5","6","7")),
  BonusMalusGLM = as.integer(pmin(BonusMalus, 150)),
  DensityGLM = as.numeric(Density),
  VehAgeGLM = relevel(VehAgeGLM, ref = "2"),   
  DrivAgeGLM = relevel(DrivAgeGLM, ref = "5"),
  Region = relevel(Region, ref = "R24")
)


We remark that for categorical variables we use the data type factor in R. This data type automatically considers dummy coding in the corresponding R procedures. Categorical variables are initialized to one class (reference level). We typically initialize to the class with the biggest volume. This initialization is achieved by the command relevel, see above. This initialization does not influence the fitted means but provides a unique parametrization. See `?relevel` for further details.

## Inspect the prepared dataset


In [ ]:
knitr::kable(head(dat2))



In [ ]:
str(dat2)



In [ ]:
summary(dat2)



# Modeling

With the prepared dataset ready, we are ready for the modeling part.

One of the frequent mistakes is to do the pre-processing after the split or inconsistently between various model to be compared. This results in not a fair comparison of the model performance.

In the following, we will fit various claim frequency models based on a Poisson assumption, to be more precise we make the following assumptions:

![1_desc](Figure_ModelAssumptions.PNG)

A priori, there is not sufficient information about this data to do a sensible decision about the best consideration of the exposure measure, either as feature or as offset. In the following we treat the exposure $v_i$ as offset to be consistent.

## Split train and test data

First, we split the dataset into train and test. Due to the potential grouping of rows in policies we can not just do a random split. For this purpose, we use the function `partition(...)` from the `splitTools` package.


In [ ]:
ind <- partition(dat2[["group_id"]], p = c(train = 0.8, test = 0.2), 
                 seed = seed, type = "grouped")
train <- dat2[ind$train, ]
test <- dat2[ind$test, ]


It describes our choices of the learning data set $\mathcal{D}$ and the test data set $\mathcal{T}$ That is, we allocate at random 80% of the policies to $\mathcal{D}$ and the remaining 20% of the policies to $\mathcal{T}$.

Usually, an 90/10 or 80/20 is used for training and test data. This is a rule-of-thumb and best practice in modeling. A good explanation can be found [here](https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio), citing as follows: "There are two competing concerns: with less training data, your parameter estimates have greater variance. With less testing data, your performance statistic will have greater variance. Broadly speaking you should be concerned with dividing data such that neither variance is too high, which is more to do with the absolute number of instances in each category rather than the percentage."

**Exercise:** Change the split from 90%/10% to 80%/20% to compare the results. If you use a split like 50%/50% the results are much worse on the test data set.

**Exercise:** Check how the final results differ if a different seed is used.


In [ ]:
# size of train/test
n_l <- nrow(train)
n_t <- nrow(test)
sprintf("Number of observations (train): %s", n_l)
sprintf("Number of observations (test): %s", n_t)

# Claims frequency of train/test
sprintf("Empirical frequency (train): %s", round(sum(train$ClaimNb) / sum(train$Exposure), 4))
sprintf("Empirical frequency (test): %s", round(sum(test$ClaimNb) / sum(test$Exposure), 4))


In [ ]:
# exposure and number of claims of train/test
# see https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3164764, p. 11 (figures do not match)
train1 <- train %>% group_by(ClaimNb) %>% summarize(n = n(), exp = sum(Exposure))
print(train1)
print(round(100 * train1$n / sum(train1$n), 3))

test1 <- test %>% group_by(ClaimNb) %>% summarize(n = n(), exp = sum(Exposure))
print(test1)
print(round(100 * test1$n / sum(test1$n), 3))


## Store model results

As we are going to compare various models, we create a table which stores the metrics we are going to use for the comparison and the selection of the best model.


In [ ]:
# table to store all model results for comparison
df_cmp <- tibble(
  model = character(),
  run_time = numeric(),
  parameters = numeric(),
  aic = numeric(),
  in_sample_loss = numeric(),
  out_sample_loss = numeric(),
  avg_freq = numeric()
)


**Exercise:** Think of other metrics to be included in the table for the model comparison, amend the table and the code below to store the new metrics in the table.

In the following, we fit and compare various claim frequency models. We compare them by using the metrics defined above.

## GLM0 (Homogeneous Model)

Let us start with the trivial model where we estimate the global mean and no features are included.

### Fitting


In [ ]:
exec_time <- system.time(glm0 <- glm(ClaimNb ~ 1, data = train, offset = log(Exposure), family = poisson()))
exec_time[1:5]
summary(glm0)


### Validation



In [ ]:
# Predictions
train$fitGLM0 <- fitted(glm0)
test$fitGLM0 <- predict(glm0, newdata = test, type = "response")
dat$fitGLM0 <- predict(glm0, newdata = dat2, type = "response")


In [ ]:
# in-sample and out-of-sample losses (in 10^(-2))
sprintf("100 x Poisson deviance GLM (train): %s", PoissonDeviance(train$fitGLM0, train$ClaimNb))
sprintf("100 x Poisson deviance GLM (test): %s", PoissonDeviance(test$fitGLM0, test$ClaimNb))

# Overall estimated frequency
sprintf("average frequency (test): %s", round(sum(test$fitGLM0) / sum(test$Exposure), 6))


In [ ]:
df_cmp[1, ] <- list("GLM0", round(exec_time[[3]], 0), length(coef(glm0)), round(AIC(glm0), 0),
                   round(PoissonDeviance(train$fitGLM0, as.vector(unlist(train$ClaimNb))), 4),
                   round(PoissonDeviance(test$fitGLM0, as.vector(unlist(test$ClaimNb))), 4),
                   round(sum(test$fitGLM0) / sum(test$Exposure), 4))
knitr::kable(df_cmp)


## GLM1 (all feature components considered)

### Fitting


In [ ]:
exec_time <- system.time(
  glm1 <- glm(ClaimNb ~ VehPowerGLM + VehAgeGLM + DrivAgeGLM + BonusMalusGLM + VehBrand +
                        VehGas + DensityGLM + Region + AreaGLM,
              data = train, offset = log(Exposure), family = poisson()))
exec_time[1:5]
summary(glm1)


A detailed analysis of the output provides that all considered features are significant, except the area code AreaGLM. This can be seen from the p-value, which is above 5%, which corresponds to "not significant".

**Exercise:** Check what happens if the same conclusion on $\texttt{AreaGLM}$ is reached, if you consider the area code as a categorical variable instead of a continuous one.

The `summary()` functions for a `glm` objects provides the statistical tests of significance for every single parameter. However, with categorical variables the primary interest is to know if a categorical variable is significant at all. This can be done using the R function `drop1`, see its help file for further details. It performs a Likelihood Ratio Test (LRT) which states that the p-value for $\texttt{AreaGLM}$ is between 1% and 5%.


In [ ]:
# needs sufficient resources!
drop1(glm1, test = "LRT")


Below we provide the sequential reduction in in-sample loss by adding one feature component after the other (ANOVA analysis). This also shows that the area code is not needed, after having already included all other feature components. From this we conclude that we may drop the
area code which is not a surprise because of the strong collinearity with the feature component Density. 

Note that the ANOVA analysis is sensitive in the order in which the feature components are considered. If we exchange the role of the area code and the density variable we obtain a similar result saying that the density variable may be dropped if the area code is already in the model.


In [ ]:
# needs sufficient resources!
anova(glm1)


**Exercise:** Extract the number of estimated coefficients from the glm object.

### Validation


In [ ]:
# Predictions
train$fitGLM1 <- fitted(glm1)
test$fitGLM1 <- predict(glm1, newdata = test, type = "response")
dat$fitGLM1 <- predict(glm1, newdata = dat2, type = "response")


In [ ]:
# in-sample and out-of-sample losses (in 10^(-2))
sprintf("100 x Poisson deviance GLM (train): %s", PoissonDeviance(train$fitGLM1, train$ClaimNb))
sprintf("100 x Poisson deviance GLM (test): %s", PoissonDeviance(test$fitGLM1, test$ClaimNb))

# Overall estimated frequency
sprintf("average frequency (test): %s", round(sum(test$fitGLM1) / sum(test$Exposure), 4))


In [ ]:
df_cmp[2, ] <- list("GLM1", round(exec_time[[3]], 0), length(coef(glm1)), round(AIC(glm1), 0),
                   round(PoissonDeviance(train$fitGLM1, as.vector(unlist(train$ClaimNb))), 4),
                   round(PoissonDeviance(test$fitGLM1, as.vector(unlist(test$ClaimNb))), 4),
                   round(sum(test$fitGLM1) / sum(test$Exposure), 4))
knitr::kable(df_cmp)


### Calibration


In addition to fitting and validating the model with a few metrics, it is important to check if the model is well calibrated across the feature space. E.g. it could be that the overall fit of a model is good, but that there are areas where the model under- and overestimates the claim frequencies. It is the goal of the subsequent calibration plots to ensure the proper fit along the whole feature space.


In [ ]:
# Area
p1 <- plot_freq(test, "AreaGLM", "frequency by area", "GLM", "fitGLM1")

# VehPower
p2 <- plot_freq(test, "VehPowerGLM", "frequency by vehicle power", "GLM", "fitGLM1")

# VehBrand
p3 <- plot_freq(test, "VehBrand", "frequency by vehicle brand", "GLM", "fitGLM1")

# VehAge
p4 <- plot_freq(test, "VehAgeGLM", "frequency by vehicle age", "GLM", "fitGLM1")

gridExtra::grid.arrange(p1, p2, p3, p4)


## GLM2 (drop feature component Area compared to GLM1)

### Fitting


In [ ]:
exec_time <- system.time(
  glm2 <- glm(ClaimNb ~ VehPowerGLM + VehAgeGLM + DrivAgeGLM + BonusMalusGLM +
                        VehBrand + VehGas + DensityGLM + Region,
              data = train, offset = log(Exposure), family = poisson()))
exec_time[1:5]
summary(glm2)


In [ ]:
# needs sufficient resources!
drop1(glm2, test = "LRT")


In [ ]:
# needs sufficient resources!
anova(glm2)


### Validation



In [ ]:
# Predictions
train$fitGLM2 <- fitted(glm2)
test$fitGLM2 <- predict(glm2, newdata = test, type = "response")
dat$fitGLM2 <- predict(glm2, newdata = dat2, type = "response")


In [ ]:
# in-sample and out-of-sample losses (in 10^(-2))
sprintf("100 x Poisson deviance GLM (train): %s", PoissonDeviance(train$fitGLM2, train$ClaimNb))
sprintf("100 x Poisson deviance GLM (test): %s", PoissonDeviance(test$fitGLM2, test$ClaimNb))

# Overall estimated frequency
sprintf("average frequency (test): %s", round(sum(test$fitGLM2) / sum(test$Exposure), 4))


In [ ]:
df_cmp[3, ] <- list("GLM2", round(exec_time[[3]], 0), length(coef(glm2)), round(AIC(glm2), 0),
                   round(PoissonDeviance(train$fitGLM2, as.vector(unlist(train$ClaimNb))), 4),
                   round(PoissonDeviance(test$fitGLM2, as.vector(unlist(test$ClaimNb))), 4),
                   round(sum(test$fitGLM2) / sum(test$Exposure), 4))
knitr::kable(df_cmp)


### Calibration



In [ ]:
# Area
p1 <- plot_freq(test, "Region", "frequency by area", "GLM", "fitGLM2")

# VehPower
p2 <- plot_freq(test, "VehPowerGLM", "frequency by vehicle power", "GLM", "fitGLM2")

# VehBrand
p3 <- plot_freq(test, "VehBrand", "frequency by vehicle brand", "GLM", "fitGLM2")

# VehAge
p4 <- plot_freq(test, "VehAgeGLM", "frequency by vehicle age", "GLM", "fitGLM2")

gridExtra::grid.arrange(p1, p2, p3, p4)


**Exercise:** Perform the calibration with other variables not yet in the charts above.

## GLM3 (drop feature components Area and VehBrand compared to GLM1)

### Fitting


In [ ]:
exec_time <- system.time(
  glm3 <- glm(ClaimNb ~ VehPowerGLM + VehAgeGLM + DrivAgeGLM + BonusMalusGLM +
                        VehGas + DensityGLM + Region,
              data = train, offset = log(Exposure), family = poisson()))
exec_time[1:5]
summary(glm3)


In [ ]:
# needs sufficient resources!
drop1(glm3, test = "LRT")


In [ ]:
# needs sufficient resources!
anova(glm3)


### Validation



In [ ]:
# Predictions
train$fitGLM3 <- fitted(glm3)
test$fitGLM3 <- predict(glm3, newdata = test, type = "response")
dat$fitGLM3 <- predict(glm3, newdata = dat2, type = "response")


In [ ]:
# in-sample and out-of-sample losses (in 10^(-2))
sprintf("100 x Poisson deviance GLM (train): %s", PoissonDeviance(train$fitGLM3, train$ClaimNb))
sprintf("100 x Poisson deviance GLM (test): %s", PoissonDeviance(test$fitGLM3, test$ClaimNb))

# Overall estimated frequency
sprintf("average frequency (test): %s", round(sum(test$fitGLM3) / sum(test$Exposure), 4))


In [ ]:
df_cmp[4, ] <- list("GLM3", round(exec_time[[3]], 0), length(coef(glm3)), round(AIC(glm3), 0),
                   round(PoissonDeviance(train$fitGLM3, as.vector(unlist(train$ClaimNb))), 4),
                   round(PoissonDeviance(test$fitGLM3, as.vector(unlist(test$ClaimNb))), 4),
                   round(sum(test$fitGLM3) / sum(test$Exposure), 4))
knitr::kable(df_cmp)


### Calibration



In [ ]:
# Region
p1 <- plot_freq(test, "Region", "frequency by area", "GLM", "fitGLM3")

# VehPowerGLM
p2 <- plot_freq(test, "VehPowerGLM", "frequency by vehicle power", "GLM", "fitGLM3")

# DriveAgeGLM
p3 <- plot_freq(test, "DrivAgeGLM", "frequency by vehicle brand", "GLM", "fitGLM3")

# VehAgeGLM
p4 <- plot_freq(test, "VehAgeGLM", "frequency by vehicle age", "GLM", "fitGLM3")

gridExtra::grid.arrange(p1, p2, p3, p4)


# Model Comparison

## Comparing metrics

We have fitted three different models, as follows:

![1_glm](Figure_glms.PNG)

We present the results of these three models below. These results are obtained by first fitting the three models to the learning data set $\mathcal{D}$, which provides the corresponding MLEs. These MLEs are then used to calculate the in-sample loss on $\mathcal{D}$. The fitted model is then applied to the testing data set $\mathcal{T}$, which provides the out-of-sample loss.


In [ ]:
knitr::kable(df_cmp)



We can draw the following conclusions:

- The first observation from the table is that the in-sample loss is smaller than the out-of-sample loss. Of course, this is not surprising because we fit on the learning data, but if this difference is too big, this may either be a sign of over-fitting or a sign that learning and test data are rather different. If the in-sample loss is larger than the out-of-sample loss, it is an indication that there are some rows which belong together and are present in both datasets.
- As stated above, the split into train and test data is highly critical in practice and is an often encountered error.
- Considering Akaike's information criterion (AIC), which introduces a penalty term for over-fitting (to mimic an out-of-sample loss), the model with the smallest AIC value should be preferred. In our case, AIC (slightly) prefers Model GLM1. However, this model has a worse out-of-sample performance than Model GLM2. Thus, we do not get a clear (and good) advise from AIC and our out-of-sample analysis here, and for later purposes we will stick to Model GLM1 as benchmark model. Note that Model GLM3 is not competitive, and the component $\texttt{VehBrand}$ is needed, in particular, for car brand $\texttt{B12}$.

## Comparing predicted claim frequency by feature level

In this section, we are going to compare the predicted claim frequency split by features. This is similar to the calibration charts above and allows a visual comparison of the models.


In [ ]:
plot_freq_2 <- function(xvar, title) {
  out <- test %>% group_by(!!sym(xvar)) %>% summarize(obs = sum(ClaimNb) / sum(Exposure),
                                             glm1 = sum(fitGLM1) / sum(Exposure),
                                             glm2 = sum(fitGLM2) / sum(Exposure),
                                             glm3 = sum(fitGLM3) / sum(Exposure))
  
  ggplot(out, aes(x = !!sym(xvar), group = 1)) + 
    geom_point(aes(y = obs, colour = "observed")) + geom_line(aes(y = obs, colour = "observed"), linetype = "dashed") +
    geom_point(aes(y = glm1, colour = "GLM1")) + geom_line(aes(y = glm1, colour = "GLM1"), linetype = "dashed") +
    geom_point(aes(y = glm2, colour = "GLM2")) + geom_line(aes(y = glm2, colour = "GLM2"), linetype = "dashed") +
    geom_point(aes(y = glm3, colour = "GLM3")) + geom_line(aes(y = glm3, colour = "GLM3"), linetype = "dashed") +
    ylim(0, 0.35) + labs(x = xvar, y = "frequency", title = title) + theme(legend.position = "bottom")
}

# Area
p1 <- plot_freq_2("Area", "frequency by Area")

# VehPower
p2 <- plot_freq_2("VehPower", "frequency by VehPower")

# VehBrand
p3 <- plot_freq_2("VehBrand", "frequency by VehBrand")

# VehAgeGLM
p4 <- plot_freq_2("VehAgeGLM", "frequency by VehAgeGLM")

gridExtra::grid.arrange(p1, p2, p3, p4)


The charts show that the predictions for area are very close for all models. For vehicle power, the models are similar but they deviate from the observation. 

We can conclude that all models provide very similar predictions, hence the best model selected above should be used.

## Comparing individual predicted claim frequency

Below we compare the out-of-sample claim frequency predictions (on log-scales) for two models. It allows to (maybe) identify if there are areas in the feature space where the predicted claim frequencies differ more/less than in other areas of the feature space.


In [ ]:
axis_min <- log(max(test$fitGLM1, test$fitGLM2))
axis_max <- log(min(test$fitGLM1, test$fitGLM2))

ggplot(test, aes(x = log(fitGLM1), y = log(fitGLM2), colour = Exposure)) + geom_point() +
  geom_abline(colour = "#000000", slope = 1, intercept = 0) +
  xlim(axis_max, axis_min) + ylim(axis_max, axis_min) +
  labs(x = "GLM1", y = "GLM2", title = "Claims frequency prediction (log-scale)") +
  scale_colour_gradient(low = "green", high = "red")


In [ ]:
axis_min <- log(max(test$fitGLM1, test$fitGLM3))
axis_max <- log(min(test$fitGLM1, test$fitGLM3))

ggplot(test, aes(x = log(fitGLM1), y = log(fitGLM3), colour = Exposure)) + geom_point() +
  geom_abline(colour = "#000000", slope = 1, intercept = 0) +
  xlim(axis_max, axis_min) + ylim(axis_max, axis_min) +
  labs(x = "GLM1", y = "GLM3", title = "Claims frequency prediction (log-scale)") +
  scale_colour_gradient(low = "green", high = "red")


# Session Info

The html is generated with the follow packages (which might be slightly newer than the ones used in the published tutorial).


In [ ]:
sessionInfo()

